Some examples and notes on getting started in Theano, loosely following the [tutorial](http://deeplearning.net/tutorial/).

In [1]:
import theano
import theano.tensor as T
from theano import function

In [2]:
x = T.dscalar('x')
y = T.dscalar('y')
z = x + y
f = function([x, y], z)

In [3]:
f(2,3)

array(5.0)

In [4]:
type(x)

theano.tensor.var.TensorVariable

In [5]:
x.type

TensorType(float64, scalar)

In [6]:
from theano import pp

In [7]:
pp(z)

'(x + y)'

In [8]:
x = T.dmatrix('x')
y = T.dmatrix('y')
z = x + y
f = function([x, y], z)

In [9]:
import numpy as np

In [10]:
f(np.array([[1,2],[3,4]]), np.array([[1,2],[3,4]]))

array([[ 2.,  4.],
       [ 6.,  8.]])

### Logistic function

In [11]:
x = T.dmatrix('x')
s = 1 / (1 + T.exp(-x))
logistic = function([x], s)
logistic([[0, 1], [-1, -2]])

array([[ 0.5       ,  0.73105858],
       [ 0.26894142,  0.11920292]])

### Default arguments

In [12]:
from theano import Param

In [13]:
x, y = T.dscalars('x', 'y')
z = x + y
f = function([x, Param(y, default=1)], z)

In [14]:
f(33)

array(34.0)

### Shared variables

In [15]:
from theano import shared

In [16]:
state = shared(0)
inc = T.iscalar('inc')
accumulator = function([inc], state, updates=[(state, state+inc)])

In [17]:
state.set_value(0)

In [18]:
accumulator(1)

array(0)

In [19]:
decrementor = function([inc], state, updates=[(state, state-inc)])

In [20]:
decrementor(1)

array(1)

In [21]:
# demonstrate use of `givens`
identity = function([inc], state, givens=[(state, inc)])

In [22]:
identity(5)

array(5, dtype=int32)

In [23]:
add1 = function([], state, updates=[(state, state+1)])

In [24]:
add1()

array(0)

### Random numbers

In [25]:
from theano.tensor.shared_randomstreams import RandomStreams

In [26]:
srng = RandomStreams(seed=123)
rv_u = srng.uniform((2,2))
rv_n = srng.normal((2,2))
f = function([], rv_u)
g = function([], rv_n, no_default_updates=True)
nearly_zeros = function([], rv_u + rv_u - 2 * rv_u)

In [27]:
f()

array([[ 0.72803009,  0.59044123],
       [ 0.23715077,  0.69958932]])

In [28]:
nearly_zeros()

array([[ 0.,  0.],
       [ 0.,  0.]])

### Monte Carlo $\pi$ estimation

In [29]:
m = 100000 # number of trials
rv_u = srng.uniform((m,))
r2 = rv_u**2 + rv_u**2
pi_est = function([], 4*T.mean(r2 < 1))

In [30]:
pi_est()

array(2.82804)

Would expect the above to work from the tutorial. Instead, seem to need:

In [31]:
m = 1000000 # number of trials
rv_u1 = srng.uniform((m,))
rv_u2 = srng.uniform((m,))
r2 = rv_u1**2 + rv_u2**2
pi_est = function([], 4*T.mean(r2 < 1))

In [32]:
pi_est()

array(3.1452199999999997)

### Logistic regression

In [33]:
from sklearn.datasets import *
from sklearn.cross_validation import train_test_split

In [34]:
data = load_iris()
Xtrain, Xtest, ytrain, ytest = (
    train_test_split(data['data'], np.where(data['target'] == 0, 0, 1)))

In [35]:
n, d = Xtrain.shape

In [36]:
alpha = 1e-3 # learning rate
lmbd = 0.05  # regularization

In [37]:
X = T.matrix('X')
y = T.vector('y')
w = shared(np.random.randn(d), name='w')
b = shared(0., name='b')

p = 1 / (1 + T.exp(-T.dot(X, w) - b))
pred = p > 0.5
xent = -y * T.log(p) - (1-y) * T.log(1-p) # cross-entropy loss
cost = xent.mean() + lmbd * (w**2).sum()
gw, gb = T.grad(cost, [w, b])

train = function(inputs=[X, y],
                 outputs=[pred, xent],
                 updates=[(w, w - alpha*gw), 
                          (b, b - alpha*gb)])

predict = function(inputs=[X], outputs=pred)

In [38]:
training_steps = 100

for i in range(training_steps):
    pred, err = train(Xtrain, ytrain)
    
(predict(Xtest) == ytest).mean()

0.97368421052631582

### Softmax regression

Generalized logisitic regression with hypothesis

$$
h_{\Theta}(x) 
= \left( \sum_{i=1}^k e^{\theta_i^T x} \right)^{-1}
\begin{bmatrix}
e^{\theta_1^T x} \\
e^{\theta_2^T x} \\
\vdots \\
e^{\theta_k^T x}
\end{bmatrix}
= \mathrm{softmax} (\Theta^{T} x)
$$

In [39]:
u = T.dvector('u')
f = function([u], T.nnet.softmax(u))

In [40]:
a = np.array([1.,2,4])
f(a)

array([[ 0.04201007,  0.1141952 ,  0.84379473]])

In [41]:
a/a.sum()

array([ 0.14285714,  0.28571429,  0.57142857])

In [42]:
from numpy.random import randn

In [43]:
# turn off optimizer for more helpful debug info
from theano import config
config.optimizer = 'None'
config.exception_verbosity = 'high'

In [213]:
class Softmax:
    
    def __init__(self, lrate=1e-3, n_iter=100):
        self.lrate = lrate
        self.n_iter = n_iter
        
    def fit(self, Xd, yd, k):
        X = T.matrix('X')
        y = T.ivector('y')
        N, d = Xd.shape
        
        self.W = shared(randn(d, k), 'W') # weights
        self.b = shared(randn(1, k), 'b', broadcastable=(True,False)) # biases
        
        # N x k matrix of hypotheses
        self.H = T.nnet.softmax(T.dot(X, self.W) + self.b)
        
        # N x 1 vector of predictions (predict class w/ highest prob.)
        y_pred = T.argmax(self.H, axis=1)
        
        # cost function (log-likelihood loss)
        cost = -T.mean(T.log(self.H)[T.arange(y.shape[0]), y])
        
        # gradient of cost function
        gW, gb = T.grad(cost, [self.W, self.b])
        
        # gradient descent updates
        updates = [(self.W, self.W - self.lrate*gW),
                   (self.b, self.b - self.lrate*gb)]
        
        self.train = function(
            inputs=[X, y],
            outputs=cost,
            updates=updates,
            allow_input_downcast=True)
        
        self.predict_proba = function(
            inputs=[X],
            outputs=self.H,
            allow_input_downcast=True)

        self.predict = function(
            inputs=[X], 
            outputs=y_pred,
            allow_input_downcast=True)
        
        for i in range(self.n_iter):
            self.train(Xd, yd)

In [214]:
Xtrain, Xtest, ytrain, ytest = (
    train_test_split(data['data'], data['target']))

In [215]:
Xtrain.shape

(112, 4)

In [216]:
ytrain.shape

(112,)

In [217]:
k = len(np.unique(ytrain))
k

3

In [220]:
clf = Softmax(n_iter=10000)
clf.fit(Xtrain, ytrain, k)
(clf.predict(Xtest) == ytest).mean()

0.97368421052631582